In [14]:
import pandas as pd
from utility_functions import get_acc, add_row_to_csv

In [15]:
dataset = 'systematic'
model = 'lstm'

In [16]:
cv_df = pd.read_csv(f"model/RNN/report_{dataset}_{model}.csv")
evaluation_df = pd.read_csv('training_data/' + dataset + '/evaluation.csv')
fold_df = pd.read_csv('training_data/' + dataset + '/folds.csv')

In [17]:
chosen_row = cv_df.loc[cv_df['val_loss'].idxmin()]
n_layer = chosen_row['num_layers']
h_size = chosen_row['hidden_size']
pred_df = pd.read_csv(f'model/RNN/predictions/{model}_{dataset}_{n_layer}layers_{h_size}features.csv')
pred_df.fillna(0, inplace=True)
eval_df = evaluation_df[evaluation_df['sequenceID'].isin(fold_df[fold_df['fold'] == 'long']['sequenceID'])]
acc = get_acc(eval_df, pred_df)
add_row_to_csv('acc_rate_csvs/' + dataset + '.csv', ['method', 'acc'], [model, acc])